In [4]:
import sys
sys.path.append('../')
import Phase_Detector.Utility as Utility
import torch

In [48]:
y_true = [1,0,0,1,0,0,1,0,0,]
y_pre_tensor = [torch.tensor([0.1,0.8]).reshape(1,2) for i in range(len(y_true))]
metric = Utility.Metrics(2,y_pred=y_pre_tensor, y_true_label=y_true)
#metric.get_accuracy()

In [52]:
metric.get_four_rate()

{'0': {'tp': tensor(0.), 'fp': tensor(0.), 'tn': tensor(3.), 'fn': tensor(6.)},
 '1': {'tp': tensor(3.), 'fp': tensor(6.), 'tn': tensor(0.), 'fn': tensor(0.)}}